In [1]:
import pandas as pd

data = pd.read_csv('creditcard_scaled.csv')
data=data.drop(['Unnamed: 0'], axis=1)
data

,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,scaled_amount,Class
0,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.783274,0
1,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.269825,0
2,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,0
3,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,0
4,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.670579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,1.034951,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.296653,0
284803,1.034963,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0.038986,0
284804,1.034975,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0.641096,0
284805,1.034975,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.167680,0


In [2]:
from sklearn.model_selection import train_test_split

#X=data.iloc[:, :-1]
#y=data.iloc[:, -1]

#X = data.loc[:, 'V1' : 'V28']
#y = data.loc[:, 'Class']

X = data.drop('Class', axis=1)
y = data['Class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)


#원본 데이터의 비율과 비슷한 비율로 나누어 졌는지 확인
print(y_train.value_counts() / y_train.count() *100 )
print(y_test.value_counts() / y_test.count() *100 )

0    99.82532
1     0.17468
Name: Class, dtype: float64
0    99.834978
1     0.165022
Name: Class, dtype: float64


In [3]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

n_inputs = X_train.shape[1]
n_outputs = 2
n_latent = 50

inputs = tf.keras.layers.Input(shape=(n_inputs, ))
x = tf.keras.layers.Dense(100, activation='tanh')(inputs)
latent = tf.keras.layers.Dense(n_latent, activation='tanh')(x)

# Encoder
encoder = tf.keras.models.Model(inputs, latent, name='encoder')
encoder.summary()

latent_inputs = tf.keras.layers.Input(shape=(n_latent, ))
x = tf.keras.layers.Dense(100, activation='tanh')(latent_inputs)
outputs = tf.keras.layers.Dense(n_inputs, activation='sigmoid')(x)

# Decoder
decoder = tf.keras.models.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

# 정상 데이터 만을 학습
x_train_norm = X_train[y_train == 0]

autoencoder = tf.keras.models.Model(inputs, decoder(encoder(inputs)))
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train_norm, x_train_norm, epochs=15, batch_size = 100, validation_data=(X_test, X_test))


Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 100)               3100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
Total params: 8,150
Trainable params: 8,150
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
________________________________________

In [4]:
encoded = encoder.predict(X_train)

'''
classifier = tf.keras.Sequential([
    layers.Dense(32, input_dim=n_latent, activation='tanh'),
    layers.Dense(16, activation='relu'),
    layers.Dense(n_outputs, activation ='softmax')
])
classifier.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.summary()
classifier.fit(encoded, y_train, batch_size=100, epochs=10)
'''

"\nclassifier = tf.keras.Sequential([\n    layers.Dense(32, input_dim=n_latent, activation='tanh'),\n    layers.Dense(16, activation='relu'),\n    layers.Dense(n_outputs, activation ='softmax')\n])\nclassifier.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\nclassifier.summary()\nclassifier.fit(encoded, y_train, batch_size=100, epochs=10)\n"

In [5]:
'''
y_pred = classifier.predict(encoder.predict(X_test))

print(accuracy_score(y_test, y_pred))


print('Accuracy Score: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('Recall Score: {:.2f}'.format(recall_score(y_test, y_pred)))
print('Precision Score: {:.2f}'.format(precision_score(y_test, y_pred)))
print('F1 Score: {:.2f}'.format(f1_score(y_test, y_pred)))
print('ROC AUC Score: {:.2f}'.format(roc_auc_score(y_test, y_pred)))
'''

"\ny_pred = classifier.predict(encoder.predict(X_test))\n\nprint(accuracy_score(y_test, y_pred))\n\n\nprint('Accuracy Score: {:.2f}'.format(accuracy_score(y_test, y_pred)))\nprint('Recall Score: {:.2f}'.format(recall_score(y_test, y_pred)))\nprint('Precision Score: {:.2f}'.format(precision_score(y_test, y_pred)))\nprint('F1 Score: {:.2f}'.format(f1_score(y_test, y_pred)))\nprint('ROC AUC Score: {:.2f}'.format(roc_auc_score(y_test, y_pred)))\n"

In [6]:
'''
from sklearn.metrics import classification_report
pred_y = classifier.predict(encoder.predict(X_test)).argmax(axis=1)
y = y_test

print(classification_report(y, pred_y))
'''

'\nfrom sklearn.metrics import classification_report\npred_y = classifier.predict(encoder.predict(X_test)).argmax(axis=1)\ny = y_test\n\nprint(classification_report(y, pred_y))\n'

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

classifiers = {
    "LogisiticRegression": LogisticRegression(max_iter=1000),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "RandomForest Classifier": RandomForestClassifier()
}

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

for key, classifier in classifiers.items():
    classifier.fit(encoded, y_train)
    training_score = cross_val_score(classifier, X_train,y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
   
    y_pred = classifier.predict(encoder.predict(X_test))
    print('Accuracy Score: {:.2f}'.format(accuracy_score(y_test, y_pred)))
    print('Recall Score: {:.2f}'.format(recall_score(y_test, y_pred)))
    print('Precision Score: {:.2f}'.format(precision_score(y_test, y_pred)))
    print('F1 Score: {:.2f}'.format(f1_score(y_test, y_pred)))
    print('ROC AUC Score: {:.2f}'.format(roc_auc_score(y_test, y_pred)))
    

Classifiers:  LogisticRegression Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.80
Precision Score: 0.91
F1 Score: 0.85
ROC AUC Score: 0.90
Classifiers:  KNeighborsClassifier Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.77
Precision Score: 0.94
F1 Score: 0.84
ROC AUC Score: 0.88
Classifiers:  SVC Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.81
Precision Score: 0.93
F1 Score: 0.86
ROC AUC Score: 0.90
Classifiers:  RandomForestClassifier Has a training score of 100.0 % accuracy score
Accuracy Score: 1.00
Recall Score: 0.80
Precision Score: 0.91
F1 Score: 0.85
ROC AUC Score: 0.90
